https://keras.io/guides/functional_api/

In [1]:
# Setup
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# start by creating an input node:

inputs = keras.Input(shape=(784,))
print(inputs.shape)
print(inputs.dtype)

(None, 784) <dtype: 'float32'>
